In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path.append('examples')

import jhutil; jhutil.color_log(1111, )

# Nerf360

In [ ]:
from pycolmap.scene_manager import SceneManager

colmap_dir = "data/360_v2/garden/sparse/0/"
manager = SceneManager(colmap_dir)
manager.load_images()

In [3]:
from jhutil.cam_viewer import get_example_datas, visualize_camera
import numpy as np
from jhutil import load_img

def qvec_to_rotmat(qvec):
	return np.array([
		[
			1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
			2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
			2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]
		], [
			2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
			1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
			2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]
		], [
			2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
			2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
			1 - 2 * qvec[1]**2 - 2 * qvec[2]**2
		]
	])

def colmap_to_c2w(qvec, tvec):
    rot = qvec_to_rotmat(-qvec)
    
    w2c = np.eye(4)
    w2c[:3, :3] = rot
    w2c[:3, -1] = tvec
    c2w = np.linalg.inv(w2c)

    c2w[0:3,2] *= -1 # flip the y and z axis
    c2w[0:3,1] *= -1
    c2w = c2w[[1,0,2,3],:]
    c2w[2,:] *= -1 # flip whole world upside down
    
    return c2w


def visualize_cam_from_manager(manager, is_colmap=False, downsample=1, show_image=True, camera_scale=1):
    transforms = []
    if show_image:
        images = []
    else:
        images = None
    for i in range(len(manager.images)):
        if i % downsample != 0:
            continue
        qvec = manager.images[i + 1].q.q
        tvec = manager.images[i + 1].t * camera_scale
        
        c2w = colmap_to_c2w(qvec.copy(), tvec.copy())
        transforms.append(c2w)
        
        if is_colmap:
            img_path = os.path.join(colmap_dir, "../../", "images_8", manager.images[i + 1].name)
        else:
            img_path = os.path.join(os.path.dirname(json_path), "images", manager.images[i + 1].name)

        if show_image:
            img = load_img(img_path)
            images.append(img)

    visualize_camera(transforms, images=images, scene_size=5)

# visualize_cam_from_manager(manager, is_colmap=True, downsample=10)

# Diva360

In [4]:
from datasets.diva360 import SceneManagerDiva360

json_path = "/data2/wlsgur4011/GESI/gsplat/data/Diva360_data/3dgs_data/penguin_0217/cameras.json"
diva_manager = SceneManagerDiva360(json_path)
diva_manager.load_cameras_from_json()
diva_manager.load_extrinsics_from_json()
diva_manager.genrate_random_points3D()

# visualize_cam_from_manager(diva_manager, is_colmap=False, downsample=1, show_image=False)

# Artemis

In [ ]:
from datasets.dfa import SceneManagerDFA

artemis_manager = SceneManagerDFA()

artemis_manager.load_cameras()
artemis_manager.load_extrinsics()
artemis_manager.genrate_random_points3D()

visualize_cam_from_manager(artemis_manager, is_colmap=False, downsample=2, show_image=True, camera_scale=3)

In [ ]:
import numpy as np
a = np.zeros((3, 4))
a[[0, 1], :]